<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

# 실습 내용

- Random Search로 Decision Tree 알고리즘 모델을 튜닝합니다.


# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [1]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%config InlineBackend.figure_format='retina'

In [2]:
# 데이터 읽어오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/income_happy.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

In [3]:
# 상위 몇 개 행 확인
data.head()

,income,happiness
0,3.862647,2.314489
1,4.979381,3.433490
2,4.923957,4.599373
3,3.214372,2.791114
4,7.196409,5.596398


In [4]:
# 기초통계량 확인
data.describe()

,income,happiness
count,498.000000,498.000000
mean,4.466902,3.392859
std,1.737527,1.432813
min,1.506275,0.266044
25%,3.006256,2.265864
50%,4.423710,3.472536
75%,5.991913,4.502621
max,7.481521,6.863388


# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) x, y 분리**

In [5]:
# target 확인
target = 'happiness'

# 데이터 분리
x = data.drop(target, axis=1)
y = data[target]

**2) 학습용, 평가용 데이터 분리**

In [6]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 데이터 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2022)

# 4.모델링

- 본격적으로 모델을 선언하고 학습하고 평가하는 과정을 진행합니다.

In [7]:
# 1단계: 불러오기
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, r2_score

In [8]:
# 2단계: 선언하기
model = DecisionTreeRegressor(max_depth=5, random_state=2022)

In [9]:
# 3단계: 학습하기
model.fit(x_train, y_train)

DecisionTreeRegressor(max_depth=5, random_state=2022)

In [10]:
# 4단계: 예측하기
y_pred = model.predict(x_test)

In [11]:
# 5단계: 평가하기
print('MAE:', mean_absolute_error(y_test, y_pred))
print('R2-Score:', r2_score(y_test, y_pred))

MAE: 0.6072155238529675
R2-Score: 0.7169483336378495


**성능에 대한 고민**

- 위 모델링 과정에서 max_depth=5를 지정했습니다.
- max_depth를 다른 값으로 지정하면 성능이 더 좋을 수도 있습니다.
- 경험이 없는 상태에서 최적의 파라미터를 지정하기는 쉽지 않습니다.


# 5.성능 튜닝


- RandomizedSearchCV를 사용해 최선의 하이퍼파라미터를 갖는 모델을 만듭니다.

## 5.1.모델 만들기

In [20]:
# 1단계: 불러오기
from sklearn.model_selection import RandomizedSearchCV

- 다음 두 가지 모델을 선언합니다.
    - 기본 모델: 기본 알고리즘을 사용하는 튜닝 대상 모델
    - 최종 모델: RandomizedSearchCV 알고리즘을 사용하는 모델
- 다음 정보를 최종 모델에 파라미터로 전달합니다.
    - 기본 모델 이름
    - 파라미터 변수
    - cv: K-Fold 분할 개수(기본값=5)
    - n_iter: 시도 횟수(기본값=10)
    - scoring: 평가 방법

In [21]:
# 2단계: 선언하기
model_dst = DecisionTreeRegressor(random_state=2022)

# 파라미터 선언
  # max_depth: 1~50
param = {'max_depth': range(1, 51)}

# Random Search 선언
  # cv=5
  # n_iter=30
  # scoring='r2'
model = RandomizedSearchCV(model_dst,   # 최적화 대상 모델
                           param,       # 사용할 파라미터
                           cv=5,        # K-Fold CV 분할 개수
                           n_iter=30,   # 랜덤하게 실행할 횟수
                           scoring='r2' # 평가 방법
                           )

In [22]:
# 3단계: 학습하기
model.fit(x_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=2022),
                   n_iter=30, param_distributions={'max_depth': range(1, 51)},
                   scoring='r2')

## 5.2.결과 확인

- model.cv_results_ 속성에 성능 테스트와 관련된 많은 정보가 포함되어 있습니다.
- 데이터프레임으로 만들어 확인하면 좋습니다.

In [24]:
# 데이터프레임 만들기
result = pd.DataFrame(model.cv_results_)
result.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002801,0.000748,0.002001,0.000632,4,{'max_depth': 4},0.715698,0.725521,0.669404,0.710552,0.469822,0.658199,0.096115,3
1,0.003200,0.000980,0.002401,0.000490,28,{'max_depth': 28},0.450496,0.516431,0.504105,0.434915,0.207685,0.422727,0.111862,14
2,0.003600,0.001357,0.001201,0.000399,7,{'max_depth': 7},0.558928,0.583268,0.639192,0.585543,0.358023,0.544991,0.097092,4
3,0.002601,0.000800,0.002201,0.000400,27,{'max_depth': 27},0.450496,0.516431,0.504105,0.434915,0.207685,0.422727,0.111862,14
4,0.003001,0.001096,0.002601,0.000800,45,{'max_depth': 45},0.450496,0.516431,0.504105,0.434915,0.207685,0.422727,0.111862,14


In [29]:
# 중요 정보 확인
result.sort_values(by='mean_test_score', ascending=False, inplace=True)
result.reset_index(drop=True, inplace=True)
result[['param_max_depth','mean_test_score']].head(10)

,param_max_depth,mean_test_score
0,3,0.677747
1,2,0.668363
2,4,0.658199
3,7,0.544991
4,1,0.535510
5,8,0.523311
6,10,0.470557
7,11,0.449955
8,15,0.427627
9,16,0.424577


- model.best_params_, model.best_score_ 값이 최적의 파라미터, 성능입니다.

In [30]:
# 선택된 파라미터와 성능 확인
print('최적 파라미터값: ', model.best_params_)
print('성능: ', model.best_score_)

최적 파라미터값:  {'max_depth': 3}
성능:  0.6777470417542905


In [34]:
# 최적의 파라미터 값을 갖는 모델
model.best_estimator_

DecisionTreeRegressor(max_depth=3, random_state=2022)

## 5.3.예측과 평가

- 모델이 최적 파라미터 값으로 학습된 상태이니 바로 예측과 평가가 가능합니다.
- 학습을 통해 예상한 성능과 실제 평가에 따른 성능은 차이가 있을 수 있습니다.
- 예선전에서 성적이 좋았다고 본선에서도 성적이 좋다고 보장할 수 없는 없겠지요?

In [31]:
# 4단계: 예측하기
y_pred = model.predict(x_test)

In [33]:
# 5단계: 평가하기
print('R2: ', r2_score(y_test, y_pred))

R2:  0.7421459185473486
